# Draw me a MOC

## The imports are a bit different in pyodide

In [ ]:
import micropip
from pathlib import Path

In [ ]:
whl_paths = list(Path.cwd().glob(pattern="pyodide/wheels/*"))
whl_paths

In [ ]:
await micropip.install("emfs:" + str(path) for path in whl_paths)
await micropip.install("widgetsnbextension")

In [ ]:
%pip install ipyaladin

In [ ]:
from ipyaladin import Aladin
from ipywidgets import Layout, Box, widgets
from astropy.coordinates import SkyCoord
from astropy.table import QTable
import astropy.units as u
from mocpy import MOC

## But then you can work like in a "normal" python

Let's print an Aladin widget

In [ ]:
aladin = Aladin(layout=Layout(width="70%"), target="M 36", fov=0.3)
info = widgets.HTML()


box_layout = Layout(
    display="flex", flex_flow="row", align_items="stretch", width="100%"
)
box = Box(children=[aladin, info], layout=box_layout)
box

In [ ]:
list_ra = []
list_dec = []
info.value = ""
i = 0
def grab_my_clicks(data):
    global list_ra
    global list_dec
    global info
    global i
    i += 1
    ra = data["ra"]
    dec = data["dec"]
    list_ra += [ra]
    list_dec += [dec]
    info.value += "<br />" + str((ra, dec))
    
aladin.add_listener("click", grab_my_clicks)

It might look like the last cell did nothng. Que nenni! The Aladin widget is waiting for you to click on the view. This will generate a list of coordinates.

That list can for example be used to generate a MOC and print it:

In [ ]:
polygon_coordinates = SkyCoord(list_ra*u.deg, list_dec*u.deg)

In [ ]:
polygon_moc = MOC.from_polygon_skycoord(polygon_coordinates, max_depth=20)

In [ ]:
aladin.add_moc_from_dict(polygon_moc.serialize("json"), {"color": "pink", "opacity": 0.5})